# Модуль ML-6, Задание 9.5
## Отбор признаков
### Крючков А.В. Группа DSPR178

In [72]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

# Загрузка данных

In [73]:
#%%capture
#!wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

In [74]:
data = pd.read_excel('data_ford_price.xlsx') 

#  Отбор признаков: мотивация

## Предобработка данных

In [75]:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

y = data['price']
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

## Обучение модели

In [76]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4682.957


## Удаление избыточного признака

In [77]:
x.drop('lat', axis = 1, inplace = True)

In [78]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [79]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4672.930


#  Отбор признаков: классификация методов

## Метод рекурсивного исключения признаков

In [80]:
from sklearn.feature_selection import RFE

In [81]:
y = data['price']
x = data.drop(columns='price')

In [82]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [83]:
estimator = LinearRegression()
selector_rfe = RFE(estimator, n_features_to_select=3, step=1)
selector_rfe = selector_rfe.fit(X_train, y_train)
 
selector_rfe.get_feature_names_out()

array(['year', 'cylinders', 'lat'], dtype=object)

In [84]:
X_train.columns

Index(['year', 'cylinders', 'odometer', 'lat', 'long', 'weather'], dtype='object')

In [85]:
selector_rfe.ranking_

array([1, 1, 4, 1, 3, 2])

##  Методы выбора признаков на основе фильтров

In [86]:
from sklearn.feature_selection import SelectKBest, f_regression

In [87]:
selector_skb = SelectKBest(f_regression, k=3)
selector_skb.fit(X_train, y_train)
 
selector_skb.get_feature_names_out()

array(['year', 'cylinders', 'odometer'], dtype=object)

# ОБУЧЕНИЕ МОДЕЛИ И ПРЕДСКАЗАНИЕ ЦЕЛЕВОГО ПРИЗНАКА ПОСЛЕ ОТБОРА ПРИЗНАКОВ

## Обучение модели и предсказание целевог опризнака после отбора по методу рекурсивного исключения признаков

In [88]:
columns_rfe = list(selector_rfe.get_feature_names_out())
X_train_rfe = X_train[columns_rfe]; X_test_rfe = X_test[columns_rfe]

In [89]:
model_rfe = LinearRegression()
model_rfe.fit(X_train_rfe, y_train)
y_predicted_rfe = model_rfe.predict(X_test_rfe)
 
mae_rfe = mean_absolute_error(y_test, y_predicted_rfe)
print('MAE RFE: %.3f' % mae_rfe)

MAE RFE: 5096.570


## Обучение модели и предсказание целевог опризнака после отбора по методу выбора признаков на основе фильтров

In [90]:
columns_skb = list(selector_skb.get_feature_names_out())
X_train_skb = X_train[columns_skb]; X_test_skb = X_test[columns_skb]

In [91]:
model_skb = LinearRegression()
model_skb.fit(X_train_skb, y_train)
y_predicted_skb = model_skb.predict(X_test_skb)
 
mae_skb = mean_absolute_error(y_test, y_predicted_skb)
print('MAE SelectKBest: %.3f' % mae_skb)

MAE SelectKBest: 4708.946


# Выводы
Метод рекурсивного исключения признаков (RFE) показал худшие результаты в метрике MAE - 5096.570 по сравнению с методом выбора признаковт на основе фильтров SelectKBest MAE - 4708.946. 

Однако, оба метода показали худшие рузультаты по сравнению обычным предсказанием без отбора MAE - 4672.930.